In [1]:
import cudaq
import numpy as np

# To model quantum noise, we need to utilise the density matrix simulator target.
cudaq.set_target("density-matrix-cpu")

In [2]:
# Let's define a simple kernel that we will add noise to.
qubit_count = 2

kernel = cudaq.make_kernel()
qubits = kernel.qalloc(qubit_count)

kernel.x(qubits[0])
kernel.x(qubits[1])

In [3]:
# In the ideal noiseless case, we get |11> 100% of the time.

ideal_counts = cudaq.sample(kernel, shots_count=1000)
print(ideal_counts)

{ 11:1000 }



In [4]:
# First, we will define an out of the box noise channel. In this case,
# we choose depolarization noise. This depolarization will result in
# the qubit state decaying into a mix of the basis states, |0> and |1>,
# with our provided probability.
error_probability = 0.1
depolarization_channel = cudaq.DepolarizationChannel(error_probability)

In [5]:
# We can also define our own, custom noise channels through
# Kraus Operator's. Here we will define two operators repsenting
# bit flip errors.

# Define the Kraus Error Operator as a complex ndarray.
kraus_0 = np.sqrt(1 - error_probability) * np.array([[1.0, 0.0], [0.0, 1.0]],
                                                    dtype=np.complex128)
kraus_1 = np.sqrt(error_probability) * np.array([[0.0, 1.0], [1.0, 0.0]],
                                                dtype=np.complex128)

In [6]:
# Add the Kraus Operator to create a quantum channel.
bitflip_channel = cudaq.KrausChannel([kraus_0, kraus_1])

# Define an empty Noise Model to add the channels to
noise_model = cudaq.NoiseModel()

In [7]:
# Apply the depolarization channel to any X-gate on the 0th qubit.
noise_model.add_channel("x", [0], depolarization_channel)
# Apply the bitflip channel to any X-gate on the 1st qubit.
noise_model.add_channel("x", [1], bitflip_channel)

In [8]:
# Due to the impact of noise, our measurements will no longer be uniformly
# in the |11> state.
noisy_counts = cudaq.sample(kernel, noise_model=noise_model, shots_count=1000)
print(noisy_counts)

{ 11:831 10:98 01:65 00:6 }

